# Get list of IPOs for past 12 months
Get the list of all the IPO stocks for the past 12 months

All I really need are the symbols. Once I have them, save them to a csv file which will be used in another script for analysis

I found a website that maintains a list for the past 12 months. I couldn't call it with an API to get it in a nice JSON format so had to scrape it in HTML and massage the HTML to get the right data. If they change the format this will need to be reworked, but this is a good starting point.

The output is a simple csv file with the list of symbols.

In [ ]:
import csv,datetime,requests
from lxml import html,etree


In [ ]:
# found a website that lists all IPOs for past 12 months. Scrape it to html then fight to get the data out. Ouch
headers = {'Content-type': 'application/json', 'Accept': 'application/json'}

response = requests.get('https://www.iposcoop.com/last-12-months/',
                         headers=headers)
print("Status code: ", response.status_code)

    

In [ ]:
# turn the html into an xml tree
tree = html.fromstring(response.content)


In [ ]:
# what does it look like in xml
print(etree.tostring(tree,pretty_print=True).decode('utf-8'))

In [ ]:
# the content I'm looking for is in the class responsive-table/table/tbody and all the tr tags
mc = tree.xpath('//*[@class="responsive-table"]/table/tbody/tr')
mc

In [ ]:
# what doe all those tr tags look like?
print(etree.tostring(mc[0],pretty_print=True).decode('utf-8'))

In [ ]:
# dig through them an look at the names
for tr in mc:
    print(tr[0][0].text)

In [ ]:
# dig through and get the symbols - this is what I really want
for idx,tr in enumerate(mc):
    print("{}  {}".format(idx,tr[1][0].text))


In [ ]:
# turn those into a simple python list
syms = [a.text for tr in mc for a in tr[1]] 
syms

In [ ]:
# want it as a name and symbol list for csv output
name_sym = [{"symbol":a.text, "name":b.text} for tr in mc for a in tr[1] for b in tr[0]]
name_sym

In [ ]:
# need a year/month string for the file name output
now = datetime.date.today()
filename = "{}_yearipos.csv".format(now.strftime("%Y%m"))
filename

In [158]:
# save as a csv for now
with open(filename, 'w') as csvfile:
    writer = csv.DictWriter(csvfile, lineterminator="\n", fieldnames=name_sym[0].keys())
    writer.writeheader()
    for data in name_sym:
        writer.writerow(data)